# Recommandation engine

This notebook was realized with python 3.6, and requires the files *P_als.txt* and *Q_als.txt* to be in the same repertory.

### First, a few imports

In [2]:
#File import + extraction
import requests, zipfile, io
from movielensutils import*

#math tools
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad
import numpy as np

#time
from time import time
from progress_bar import InitBar

## 1) Introducing the model

In [3]:
#Retrieving the dataset
zip_url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_file = requests.get(zip_url)
zip_file = zipfile.ZipFile(io.BytesIO(zip_file.content))
zip_file.extractall()

In [16]:
#Loading the dataset
R, mask = load_movielens("ml-100k/u.data")
print("There are %d users and %d movies in the database"%(R.shape[0], R.shape[1]))
print("Total amount of rankings n = %d "%(np.count_nonzero(mask)))

There are 943 users and 1682 movies in the database
Total amount of rankings n = 100000 


The objective function to be minimized is
$$ g : P,Q \mapsto \frac{1}{2}\left|\left|
\mathbb{1}_K \circ (R - QP) \right|\right|_F^2 + \frac{\rho}{2}\left|\left|
Q\right|\right|_F^2 + + \frac{\rho}{2}\|
P \|_F^2$$ 
The gradient is 
$$ \bigtriangledown g : P,Q \mapsto \begin{pmatrix}
                    - Q^T\left[\mathbb{1}_K\circ (R - QP)\right] + \rho P \\ 
                    - \left[\mathbb{1}_K\circ (R - QP)\right] P^T + \rho Q
                    \end{pmatrix}
$$ 

The Hessian matrix is... painful to interpret. To answer the question: "is it a convex function?" so that we know whether a minimum is global or not, we will try another approach. As the product QP is a good sign that it is NOT convex, we will show that the Hessian related to the corresponding 1D problem is not defined positive.

Let us set $\hat{g}(p,q) = \frac{1}{2}(r-pq)^2$ with $\left(r,p,q\right) \in \mathbb{R}^3$ (because $\frac{\rho}{2}\left(q^2+p^2\right)$ is convex anyway).
Then we have $\triangledown \hat{g}(p,q) =
\begin{pmatrix}
                    - q(r-pq) \\ 
                    - p(r-pq)
\end{pmatrix}$
$\triangledown^2 \hat{g}(p,q) =
\begin{pmatrix}
                    q^2 && 2qp - r \\ 
                    2qp-r &&p^2
\end{pmatrix}$

Therefore $\begin{pmatrix}p & q \end{pmatrix}\triangledown^2 \hat{g}\begin{pmatrix} p \\ q \end{pmatrix} = (px_1+qx_2)^2+2x_1x_2(qp-r)$. This not always positive, even with $x \neq 0$. For example, with the set of values $\big(x_1, x_2, p, q, r\big) = \big(1,1,1,1,3\big)$, it is equal to 0, and with $r = 4$ instead, it is negative.

$g$ is not lipschitzian because the hessian matrix coefficients are not limited to a specific range, but a polynomial expression of P and Q with degre = 2... It is therefore not lipschitzian.

## 2) Find $P$ when $Q_0$ is fixed

The function $ g : P \mapsto \frac{1}{2}\left|\left|
\mathbb{1}_K \circ (R - Q^0P) \right|\right|_F^2 + \frac{\rho}{2}\left|\left|
Q^0\right|\right|_F^2 + + \frac{\rho}{2}\|
P \|_F^2$ has for gradient $\bigtriangledown g(P) = -{Q^0}^T\mathbb{1}_K \circ (R - Q^0P) + \rho P$ and for hessian matrix $\bigtriangledown^2 g(P) = {Q^0}^T \mathbb{1}_K \mathbb{1}_K^T {Q^0} + \rho I$ which is defined positive when $\rho$ is carefully chosen. Therefore g is convex.

**BEWARE** : the computing time of the next cell may take up to 7 min (check grad takes a while). I prefered to use the full dataset

In [17]:
def objective(P, Q0, R, mask, rho):
    """
    g in the problem (objective function)
    
    Input:
    P : Matrix input with shape C x I
    Q0 : matrix with shape  U x C
    R : matrix with shape U x I
    mask : binary matrix with shape U x I
    rho : strictly positive real

    Output :
    val : g(P)
    grad_P : obviously...
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    grad_P = -np.dot(Q0.T, tmp) + rho*P
    return (val, grad_P)


#Initialization of the parameters (R and mask are already defined in the previous cell)
C = 6
U, I = R.shape
Q0,_,P0 = svds(R, k=C)
rho = 0.2

#Functions required for use in the check_grad function
def func_g(P):
    """
    same as objective, but the purpose is to use a vector as an argument and return only val
    """
    global Q0, R, mask, rho, C
    PP = np.reshape(P, (C, I))
    return objective(PP, Q0, R, mask, rho)[0]

def func_g_grad(P):
    """
    same as objective, but the purpose is to use a vector as an argument and return only grad_P
    """
    global Q0, R, mask, rho, C
    PP = np.reshape(P, (C, I))
    return np.ravel(objective(PP, Q0, R, mask, rho)[1])

#checking the gradiant precision
error = check_grad(func_g, func_g_grad, np.ravel(P0))
relative_error = error / np.linalg.norm(objective(P0, Q0, R, mask, rho)[1])
print(error, relative_error)

1.16737625347 0.00152389552688


The error could be lower, but it is still ok as the relative error is low.

Now let us code a fixed step gradient descent algorithm

In [18]:
def gradient(f_and_grad, P0, gamma, epsilon):
    """
    executes a gradient descent with fixed step
    
    Input:
    f_and_grad is a tuple : returns both function and its gradient.
    P0 = initialization point
    epsilon : threshold for gradient variation
    """
    P = P0.copy()
    n = 0
    grad = f_and_grad(P)[1]
    while np.linalg.norm(grad) > epsilon and n < 300 :
        grad = f_and_grad(P)[1]
        P = P - gamma*grad
        n+=1
    return P,n    

#Computing the constant from the lipschitzian function inequality
L = rho + np.linalg.norm(np.dot(Q0.T, Q0))

t = time()
Pmin, n = gradient(lambda P : objective(P, Q0, R, mask, rho), P0, 1/L, 1.)
print("Fixed step gradient descent : %d steps, time elapsed %0.3f s"%(n, time()-t))

Fixed step gradient descent : 43 steps, time elapsed 1.854 s


## 3) Algorithmical optimization for the problem with $Q_0$ fixed

We can use the conjugate gradient method here because the function $g$ is a quadratic form : 

indeed, $ g(P) = \frac{1}{2}\left|\left|
\mathbb{1}_K \circ (R - Q^0P) \right|\right|_F^2 + \frac{\rho}{2}\left|\left|
Q^0\right|\right|_F^2 + \frac{\rho}{2}\|
P \|_F^2 = \frac{1}{2} \left( \left(\mathbb{1}_K \circ (R - Q^0P)\right)^T\mathbb{1}_K \circ(R - Q^0P)\right) + \frac{\rho}{2}\left( {Q^0}^TQ^0 \right) + \frac{\rho}{2}\left( P^TP \right) = \frac{1}{2} P^T \left( {Q^0}^T \mathbb{1}_K \mathbb{1}_K^T {Q^0} + \rho I\right) P - \frac{1}{2} \left( P^T{Q^0}^T(\mathbb{1}_K \circ R) + (\mathbb{1}_K \circ R)^TQP \right) + \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q^0}^TQ^0\right)$

Therefore, $g(P) = \frac{1}{2}P^TAP + b^TP + P^Tb + c$ with 
$$\begin{align*} 
A &= {Q^0}^T (\mathbb{1}_K \mathbb{1}_K^T){Q^0} + \rho I \\ 
b &= -\frac{1}{2} {Q^0}^T(\mathbb{1}_K \circ R)
\\ c &= \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q^0}^TQ^0\right)
\end{align*}$$

Which provides a gradient $\triangledown g(P) = AP + b +b^T$, and A being ysmetrical, positive definite, it allows for the application of *Fletcher & Reeves* algorithm.

In [22]:
#tool  for Linear gradient descent : find slope.
def find_gamma(P, f_and_grad, a, b):
    k = 0
    lgrad = f_and_grad(P)[1]
    lfunc = f_and_grad(P)[0]
    gamma = b*(a**k)
    PP = P - gamma*lgrad
    # other possible condition : f_and_grad(PP)[0] >= lfunc - 0.5*gamma*(np.linalg.norm(P-PP)**2)
    while f_and_grad(PP)[0] >= lfunc + np.vdot(lgrad, (PP-P)) + 1/(2*gamma)*(np.linalg.norm(P-PP)**2) :
        k += 1
        gamma = b*(a**k)
        PP = P - gamma*lgrad
    return b*(a**(k-1))

#Linear gradient descent using Armijo's line search
def gradient_lin(f_and_grad, P0, epsilon, a=0.5, bb=0.5):
    """
    executes a gradient descent with linear step
    
    Input:
    f_and_grad is a tuple : returns both function and its gradient.
    P0 = initialization point
    epsilon : threshold for gradient variation
    """
    P = P0.copy()
    n = 0
    gamma = bb/2
    grad = f_and_grad(P)[1]
    while np.linalg.norm(grad) > epsilon and n < 300 :
        grad = f_and_grad(P)[1]
        b = 2*gamma
        gamma = find_gamma(P, f_and_grad, a, b)
        P = P - gamma*grad
        n+=1
    return P, n 

# def gradient_conj(f_and_grad, A, P0, epsilon):
#     """
#     executes a gradient descent with Fletcher & Reeves technique
    
#     Input:
#     f_and_grad is a tuple : returns both function and its gradient.
#     A = matrix in the standard expression of the quadratic form g
#     P0 = initialization point
#     epsilon : threshold for gradient variation
#     """
#     P = P0
#     n = 0
#     g = f_and_grad(P)[1]
#     d = - g
#     s = - np.dot(d.T, g) / np.dot(d.T, np.dot(A, d)) #PROBLEM HERE as the dividend is not supposed to be a matrix
#     P = P + np.dot(d,s)
#     while np.linalg.norm(d) > epsilon and n < 300 :
#         g = f_and_grad(P)[1]
#         b = np.dot(d.T, np.dot(A, g)) / np.dot(d.T, np.dot(A, d))
#         d = - g + np.dot(d,b)
#         s = - np.dot(d.T, g) / np.dot(d.T, np.dot(A, d))
#         P = P + np.dot(d,s)
#         n+=1
#     return P,n

#The matrix A in the standard expression of the quadratic form g
# A = 2*np.dot(Q0.T, np.dot(np.dot(mask, mask.T), Q0)) + rho*np.eye(C)
# b = -0.5* np.dot(Q0.T, R*mask)
# A = 2*np.dot(Q0.T, Q0) + rho*np.eye(C)

#useful in conjugate gradient descent method
def linear_operator(d, Q, mask):
    tmp = (Q.dot(d)) * mask
    Ad = rho * d + Q.T.dot(tmp)
    return Ad

def gradient_conj(f_and_grad, lin_op, P0, epsilon, max_iter=np.inf):
    """
    executes a gradient descent with Fletcher & Reeves technique
    
    Input:
    f_and_grad is a tuple : returns both function and its gradient.
    lin_op = matrix in the standard expression of the quadratic form g, 
             adapted to the current problem where P is no longer a vector
    P0 = initialization point
    epsilon : threshold for gradient variation
    max_iter : when to stop if gradient does not get below epsilon fast enough
    """
    P = P0.copy() 
    n = 0

    val, grad = f_and_grad(P)
    d = - grad
    Ad = lin_op(d)
    s = - np.sum(d * grad) / np.sum(d * Ad)
    P = P + s * d

    while np.linalg.norm(grad) > epsilon and n <max_iter:
        val, grad = f_and_grad(P)

        b = np.sum(Ad * grad) / np.sum(d * Ad)
        d = -grad + b * d
        Ad = lin_op(d)
        s = - np.sum(d * grad) / np.sum(d * Ad)
        P = P + s * d

        n += 1

    return P, n

#Comparison of the time necessary for each technique to reach its solution
t = time()
Pmin, n = gradient(lambda P : objective(P, Q0, R, mask, rho), P0, 1/L, 1.)
print("Fixed step gradient descent : %d steps, time elapsed %0.3f s"%(n, time()-t))
t = time()
Pmin2, n = gradient_lin(lambda P : objective(P, Q0, R, mask, rho), P0, 1.)
print("Linear gradient descent : %d steps, time elapsed %0.3f s"%(n, time()-t))
t = time()
Pmin3, n = gradient_conj(lambda P : objective(P, Q0, R, mask, rho), lambda d: linear_operator(d, Q0, mask), P0, 1.)
print("Conjugated gradients descent : %d steps, time elapsed %0.3f s"%(n, time()-t))

#comparison of the results
print("\n")
print("Relative error between fs and lin = %0.3f %%" % (np.linalg.norm(Pmin-Pmin2)/np.linalg.norm(Pmin)*100))
print("Relative error between fs and conj = %0.3f %%" % (np.linalg.norm(Pmin-Pmin3)/np.linalg.norm(Pmin)*100))
print('\n')
print("objective g(P,Q0) = %0.3f for fixed step method"%(objective(Pmin, Q0, R, mask, rho)[0]))
print("objective g(P,Q0) = %0.3f for linear method"%(objective(Pmin2, Q0, R, mask, rho)[0]))
print("objective g(P,Q0) = %0.3f for conjugate method"%(objective(Pmin3, Q0, R, mask, rho)[0]))

Fixed step gradient descent : 43 steps, time elapsed 1.899 s
Linear gradient descent : 11 steps, time elapsed 2.652 s
Conjugated gradients descent : 7 steps, time elapsed 0.518 s


Relative error between fs and lin = 0.278 %
Relative error between fs and conj = 0.332 %


objective g(P,Q0) = 303938.100 for fixed step method
objective g(P,Q0) = 303936.658 for linear method
objective g(P,Q0) = 303936.586 for conjugate method


As a result, the most efficient techniques, as measured per the number of steps are in order : the conjugated gradients descent, the linear gradient descent, and the fixed step gradient descent. but linear gradient descent seems to be more time consuming despite that.


## 4) Complete Solution

When using the linear gradient descent technique, the output is a local minimum, not necessarily the global optimum as $g : (P,Q) \mapsto g(P,Q)$ is not convex.

In [23]:
#same as objecive, but taking Q into account
def total_objective(P, Q, R, mask, rho):
    """
    g in the problem (objective function)
    
    Input:
    P : Matrix input with shape C x I
    Q : matrix with shape  U x C
    R : matrix with shape U x I
    mask : binary matrix with shape U x I
    rho : strictly positive real

    Output :
    val : g(P, Q)
    grad_P : obviously...
    grad_Q : obviously...
    """

    tmp = (R - Q.dot(P)) * mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))
    grad_P = -np.dot(Q.T, tmp) + rho*P
    grad_Q = -np.dot(tmp, P.T) + rho*Q
    
    return val, grad_P, grad_Q

#same as above, but vectorized, so that we can use the linear gradient solver
def total_objective_vectorized(PQvec, R, mask, rho):
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] / (n_items + n_users)
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))

    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])

t = time()
PQvec_sol, n = gradient_lin(lambda t : total_objective_vectorized(t, R, mask, rho), np.concatenate([P0.ravel(), Q0.ravel()]), 100)

n_items = R.shape[1]
n_users = R.shape[0]
F = PQvec_sol.shape[0] / (n_items + n_users)
Pvec_sol = PQvec_sol[0:n_items*F]
Qvec_sol = PQvec_sol[n_items*F:]
P_sol = np.reshape(Pvec_sol, (F, n_items))
Q_sol = np.reshape(Qvec_sol, (n_users, F))

print("result of the complete problem obtained after %d steps, %0.3f s"%(n, time()-t))
relative_error_sol = np.linalg.norm(R - np.dot(Q_sol, P_sol))/np.linalg.norm(R)*100
print("relative error = %0.0f %%"% relative_error_sol)
print("value of the objective function g(P,Q) = %0.5f"%(total_objective(P_sol, Q_sol, R, mask, rho)[0]))

C:\Users\Benjamin\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Benjamin\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Benjamin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)


result of the complete problem obtained after 210 steps, 57.875 s
relative error = 263 %
value of the objective function g(P,Q) = 32741.31229


C:\Users\Benjamin\Anaconda3\lib\site-packages\ipykernel\__main__.py:45: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Benjamin\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### Alternate Least Square solving

It converges because :
- at each step, the problem solved is convex. The $P$ or $Q$ found always exists, and makes the optimization function at least no greater than before computation.
- So after each consecutive optimization of $P$ and $Q$, the optimization function has decreased (not striclty).

As we obtain a monotonic function ($(P,Q) \mapsto g(P,Q)$ decreases), that is bounded (because g takes values in $\mathbb{R}_+$), the algorithm converges.

**BEWARE:** takes quite a while, possibly _**more than 30min**_ !!!!!!

In [186]:
def objective_Q(P0, Q, R, mask, rho):
    """
    g in the problem (objective function)
    
    Input:
    P0 : Matrix input with shape C x I
    Q : matrix with shape  U x C
    R : matrix with shape U x I
    mask : binary matrix with shape U x I
    rho : strictly positive real

    Output :
    val : g(Q)
    grad_Q : obviously...
    """

    tmp = (R - Q.dot(P0)) * mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P0 ** 2))
    grad_Q = -np.dot(tmp, P0.T) + rho*Q
    
    return (val, grad_Q)

def ALS(P0, Q0, R, mask, rho, epsilon, eps_in, N):
    from progress_bar import InitBar
    n = 0
    P, Q = P0.copy(), Q0.copy()
    _, grad_P, grad_Q = total_objective(P0, Q0, R, mask, rho)
    
#     bar = InitBar()
   
    while (np.linalg.norm(grad_P) >= epsilon or np.linalg.norm(grad_Q) >= epsilon) and n <= N:
        P,_ = gradient_lin(lambda P : objective(P, Q, R, mask, rho), P, eps_in)
        Q,_ = gradient_lin(lambda Q : objective_Q(P, Q, R, mask, rho), Q, eps_in)
        val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
        n += 1
        print(n, "/", N+1, ",  g(P,Q)= ",val, ",  grad_P = ",np.linalg.norm(grad_P), ",  grad_Q = ", np.linalg.norm(grad_Q))
#         bar(n/N*100)

#     del bar
            
    return P, Q, n

# P_als.txt and Q_als.txt are files which store the best values minimizing g obtained during a previous iteration of 
# the algorithm. Instead of using it, one can consider the values P0 and Q0 as before

t = time()
P_als, Q_als, n = ALS(np.loadtxt('P_als.txt'), np.loadtxt('Q_als.txt'), R, mask, rho, 5., 5., 30)
print("result of the complete problem obtained after %d steps, %0.3f s"%(n, time()-t))
relative_error_als = np.linalg.norm(R - np.dot(Q_als, P_als))/np.linalg.norm(R)*100
print("relative error = %0.0f %%"% relative_error_als)
print("value of the objective function g(P,Q) = %0.5f "%(total_objective(P_als, Q_als, R, mask, rho)[0]))

#storing these values so that if we restart the algorithm, we have a better starting point.
np.savetxt('P_als.txt', P_als)
np.savetxt('Q_als.txt', Q_als)

result of the complete problem obtained after 0 steps, 0.192 s
relative error = 324 %
value of the objective function g(P,Q) = 30134.47905 


### Comparison of ALS, and non convex solving using Linear Gradient Descent

**According to the information printed before**
- The computing time is much lower for LGD than for ALS
- The precision is greater for ALS than for LGD ... but it is not obvious when comparing it with the same $\epsilon = 100$ 
- The value of the objective function is lower for ALS than for LGD, but after many iterations (30 x 20)

For ALS, as we come closer to a minimum, the computing time per step decreases a little.

One can notice that the product PQ is farther from R with ALS than with LGD (this is what the relative error measures), however this is not incompatible with the fact the objective function is lower with ALS

## Recommandation for user 449

The aim is to suggest the ID of the movie, a movie he has not seen yet, to which user n°449 would give the greater rating, according to the matrix factorization $R = QP$.
$
R = \underset{ Q : \; size \; = \; U \times C}{\begin{bmatrix}
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & User \; n°499 \; appreciation \; for \; each \; characteristic \; C_U  \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
  & & \_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\ \\
 \end{bmatrix}} \times 
 \underset{ P : \; size \; = \; C \times I}{\begin{bmatrix}
  & | & | & | & Film \; n°I & | & |& |& |&\\
  & | & | & | & with \; characteristics \; C_I & | & |& |& |&\\
  & | & | & | & & |& |& |& |&\\
  & | & | & | & &|& |& |& |&\\
 \end{bmatrix}}
 $
 
So to find the movie best suited for user n°449, all we have to do is consider the line $Q_{449,:}$ which describes the appreciation for each movie characteristic ( something like drama, comedy, love story, violence, countries, etc.) and make the scalar product $\left<Q_{449,:} \;;\; P\right>^T$ which return $M_{449}$, a vector of size $I\times1$ which represents the rating that user 449 would give for each movie.

All what is left to do is selecting the movie with the highest rating, yet unseen by the user 449. Assuming there is at least one positive score in the list, it can be found by computing $M_{449}' = \left<Q_{449,:} \;;\; P\right>^T \underset{element-wise}{*} \left(1_{I \times 1}^T - \mathbb{1}_K^T \right) $ where $1$ is a matrix made esclusively of '1'. The movie to be recommanded is the index of the greater value of this vector.

In [189]:
m = np.dot(Q_als[449,:], P_als)
print("Number of negatively rated movies: %d out of %d"%(np.count_nonzero(m<0), m.shape[0]))
m = m * (1 - mask[449,:])
m = m.T
movie_index = np.argmax(m)
print("The movie I would suggest to user 449 is movie n° %d (out of the %d available ) (with rating %0.4f)"%(movie_index, m.shape[0], m [movie_index]))

# One can standardize m so as to attribute a mark, as a percentage, to each movie.
Range = m[np.argmax(m)] - m[np.argmin(m)]
m = (m - m[np.argmin(m)])/Range*100
#print("standardized rating for the movie suggested = %0.2f %% " %(m[movie_index]))
print("\n list of best rated movies by user 449, indexed by movie ID to the left,\
        and predicted rating to the right (/100)" )

list_of_best_movies = np.argsort(-m)
sorted_ratings = np.zeros((m.shape[0], 2))
sorted_ratings[:, 0], sorted_ratings[:, 1] = list_of_best_movies, m[list_of_best_movies]
    
print(np.round(sorted_ratings[:20], decimals=0).astype(int))

Number of negatively rated movies: 0 out of 1682
The movie I would suggest to user 449 is movie n° 1462 (out of the 1682 available ) (with rating 5.5224)

 list of best rated movies by user 449, indexed by movie ID to the left,        and predicted rating to the right (/100)
[[1462  100]
 [ 908   98]
 [1063   93]
 [1448   93]
 [1274   92]
 [1366   92]
 [1174   91]
 [1642   91]
 [1251   90]
 [1241   90]
 [1175   90]
 [ 962   89]
 [1425   88]
 [1168   88]
 [ 407   87]
 [ 866   87]
 [1074   87]
 [ 250   86]
 [1193   86]
 [1232   86]]


In [ ]:
#contact Roland BADEAU email telecom parisetch